# https://towardsdatascience.com/an-introduction-to-web-scraping-with-python-a2601e8619e5

In [19]:
###############################################################
###### Import necessary dependancy#######
#############################################################

import pandas as pd
import numpy as np

###############################################################
###### Import necessary dependancy#######
#############################################################

import pandas as pd
import numpy as np
###############################################################
###### Import necessary  library for  scraping more page#######
############################################################
from bs4 import BeautifulSoup # Import Beautifulsoup to scraping the web pafr
from requests import get # import get from request to get all the pa
main_url = "http://books.toscrape.com/index.html"
import requests
result = requests.get(main_url)
result.text[:1000]
###############################################################
###### Import necessary  library for  scraping more page#######
#############################################################

from bs4 import BeautifulSoup # Import Beautifulsoup to scraping the web pafr
from requests import get # import get from request to get all the pa

### Warm-up: get the content of the main page

In [8]:
main_url = "http://books.toscrape.com/index.html"

import requests
result = requests.get(main_url)
#result.text[:1000]

soup = BeautifulSoup(result.text, 'html.parser')
#print(soup.prettify()[:1000])

#### Let’s define a function to request and parse a HTML web page as we will need this a lot during this tutorial

In [23]:
def getAndParseURL(url):
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    return(soup)

BeautifulSoup enables us to find those special ‘article’ tags. We can wall the  
find() function in order to find the first occurence of this tag in the HTML:

In [9]:
soup.find("article", class_ = "product_pod")

<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">Â£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

#### Let’s dive deeper in the tree by adding the other child tags:

In [10]:
soup.find("article", class_ = "product_pod").div.a

<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>

#### We can get this by adding .get(“href”) to the previous instruction:

In [11]:
soup.find("article", class_ = "product_pod").div.a.get('href')

'catalogue/a-light-in-the-attic_1000/index.html'

#### Now let’s gather all the products URLs on the main web page at once using the findAll() function:

In [12]:
main_page_products_urls = [x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")]

print(str(len(main_page_products_urls)) + " fetched products URLs")
print("One example:")
main_page_products_urls[0]

20 fetched products URLs
One example:


'catalogue/a-light-in-the-attic_1000/index.html'

#### Now let’s use this to define a function to retrieve book links on any given page of the website:

In [29]:
def getBooksURLs(url):
    soup = getAndParseURL(url)
    # remove the index.html part of the base url before returning the results
    return(["/".join(url.split("/")[:-1]) + "/" + x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")])

#### We can tell BeautifulSoup to match the URLs that contain this pattern in order to retrieve easily the categories URLs:

In [14]:
import re

categories_urls = [main_url + x.get('href') for x in soup.find_all("a", href=re.compile("catalogue/category/books"))]
categories_urls = categories_urls[1:] # we remove the first one because it corresponds to all the books

print(str(len(categories_urls)) + " fetched categories URLs")
print("Some examples:")
categories_urls[:5]

50 fetched categories URLs
Some examples:


['http://books.toscrape.com/index.htmlcatalogue/category/books/travel_2/index.html',
 'http://books.toscrape.com/index.htmlcatalogue/category/books/mystery_3/index.html',
 'http://books.toscrape.com/index.htmlcatalogue/category/books/historical-fiction_4/index.html',
 'http://books.toscrape.com/index.htmlcatalogue/category/books/sequential-art_5/index.html',
 'http://books.toscrape.com/index.htmlcatalogue/category/books/classics_6/index.html']

#### In order to fetch all the products URLs, we need to be able to get through all the pages.  To do so, we can go iteratively through all the ‘next’ buttons.

In [24]:
# store all the results into a list
pages_urls = [main_url]

soup = getAndParseURL(pages_urls[0])

# while we get two matches, this means that the webpage contains a 'previous' and a 'next' button
# if there is only one button, this means that we are either on the first page or on the last page
# we stop when we get to the last page

while len(soup.findAll("a", href=re.compile("page"))) == 2 or len(pages_urls) == 1:
    
    # get the new complete url by adding the fetched URL to the base URL (and removing the .html part of the base URL)
    new_url = "/".join(pages_urls[-1].split("/")[:-1]) + "/" + soup.findAll("a", href=re.compile("page"))[-1].get("href")
    
    # add the URL to the list
    pages_urls.append(new_url)
    
    # parse the next page
    soup = getAndParseURL(new_url)
    

print(str(len(pages_urls)) + " fetched URLs")
print("Some examples:")
pages_urls[:5]

50 fetched URLs
Some examples:


['http://books.toscrape.com/index.html',
 'http://books.toscrape.com/catalogue/page-2.html',
 'http://books.toscrape.com/catalogue/page-3.html',
 'http://books.toscrape.com/catalogue/page-4.html',
 'http://books.toscrape.com/catalogue/page-5.html']

#### Fortunately the result of a request has a very useful attribute that can show us the return status of the HTML request

In [25]:
result = requests.get("http://books.toscrape.com/catalogue/page-50.html")
print("status code for page 50: " + str(result.status_code))

result = requests.get("http://books.toscrape.com/catalogue/page-51.html")
print("status code for page 51: " + str(result.status_code))

status code for page 50: 200
status code for page 51: 404


### We can use this information to get all our pages URLs: we should iterate until we get a 404 code.

### Let’s try this method now:

In [26]:
pages_urls = []

new_page = "http://books.toscrape.com/catalogue/page-1.html"
while requests.get(new_page).status_code == 200:
    pages_urls.append(new_page)
    new_page = pages_urls[-1].split("-")[0] + "-" + str(int(pages_urls[-1].split("-")[1].split(".")[0]) + 1) + ".html"
    

print(str(len(pages_urls)) + " fetched URLs")
print("Some examples:")
pages_urls[:5]

50 fetched URLs
Some examples:


['http://books.toscrape.com/catalogue/page-1.html',
 'http://books.toscrape.com/catalogue/page-2.html',
 'http://books.toscrape.com/catalogue/page-3.html',
 'http://books.toscrape.com/catalogue/page-4.html',
 'http://books.toscrape.com/catalogue/page-5.html']

### Get all products URLs
Now the next step consists in fetching all the products URLs for every page    
This step is quite simple as we already have the list of all pages and the function  
to get products URLs from a page.

Let’s iterate through the pages and apply our function:

In [27]:
booksURLs = []
for page in pages_urls:
    booksURLs.extend(getBooksURLs(page))
    
print(str(len(booksURLs)) + " fetched URLs")
print("Some examples:")
booksURLs[:5]

1000 fetched URLs
Some examples:


['http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html',
 'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html',
 'http://books.toscrape.com/catalogue/soumission_998/index.html',
 'http://books.toscrape.com/catalogue/sharp-objects_997/index.html',
 'http://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html']

### Get product data
The last step consist in scraping the data for each product.  
Let’s explore first how the information is structured on the products pages:

In [28]:
booksURLs = []
for page in pages_urls:
    booksURLs.extend(getBooksURLs(page))
    
print(str(len(booksURLs)) + " fetched URLs")
print("Some examples:")
booksURLs[:5]

,name,nb_in_stock,price,product_category,rating,url_img
0,A Light in the Attic,22,51.77,poetry_23,Three,http://books.toscrape.com/catalogue/a-light-in...
1,Tipping the Velvet,20,53.74,historical-fiction_4,One,http://books.toscrape.com/catalogue/tipping-th...
2,Soumission,20,50.10,fiction_10,One,http://books.toscrape.com/catalogue/soumission...
3,Sharp Objects,20,47.82,mystery_3,Four,http://books.toscrape.com/catalogue/sharp-obje...
4,Sapiens: A Brief History of Humankind,20,54.23,history_32,Five,http://books.toscrape.com/catalogue/sapiens-a-...
